In [1]:
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent))
# print(str(Path().resolve().parent))
print(sys.path)

['C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311', 'd:\\github\\scikit-quri\\pyenv', '', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages\\win32', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages\\win32\\lib', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages\\Pythonwin', 'D:\\github\\scikit-quri']


In [2]:
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent.parent))

# import better_exceptions
import numpy as np
from typing import Optional
from quri_parts.circuit.utils.circuit_drawer import draw_circuit
from quri_parts.circuit import UnboundParametricQuantumCircuit,LinearMappedUnboundParametricQuantumCircuit
from numpy.random import default_rng
from scikit_quri.circuit.circuit import LearningCircuit

from qulacs import Observable
from quri_parts.core.operator import Operator, pauli_label
from quri_parts.core.estimator.gradient import (
    create_parameter_shift_gradient_estimator,
    create_numerical_gradient_estimator,)
from quri_parts.qulacs.estimator import (
    create_qulacs_vector_concurrent_estimator,
    create_qulacs_vector_parametric_estimator,
    create_qulacs_vector_concurrent_parametric_estimator,
)
from quri_parts.algo.optimizer import Adam, LBFGS




In [3]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
x = df.loc[:, ["petal length (cm)", "petal width (cm)"]]
x_train,x_test,y_train,y_test = train_test_split(x,iris.target,test_size=0.25,random_state=0)

x_train = x_train.to_numpy()

from quri_parts.circuit.utils.circuit_drawer import draw_circuit
import matplotlib.pyplot as plt
n_qubits = 5
num_class = 3
print(x_train)

[[4.2 1.5]
 [4.  1.2]
 [5.5 2.1]
 [1.3 0.2]
 [5.1 2.3]
 [1.6 0.6]
 [1.5 0.2]
 [3.5 1. ]
 [5.5 1.8]
 [5.7 2.5]
 [5.  1.5]
 [5.8 1.8]
 [3.9 1.1]
 [6.1 2.3]
 [4.7 1.6]
 [3.8 1.1]
 [4.9 1.8]
 [5.1 1.5]
 [4.5 1.7]
 [5.  1.9]
 [4.7 1.4]
 [5.2 2. ]
 [4.5 1.6]
 [1.6 0.2]
 [5.1 1.9]
 [4.2 1.3]
 [3.6 1.3]
 [4.  1.3]
 [4.6 1.4]
 [6.  1.8]
 [1.5 0.2]
 [1.1 0.1]
 [5.3 1.9]
 [4.2 1.2]
 [1.7 0.2]
 [1.5 0.4]
 [4.9 1.5]
 [1.5 0.2]
 [5.1 1.8]
 [3.  1.1]
 [1.4 0.3]
 [4.5 1.5]
 [6.1 2.5]
 [4.2 1.3]
 [1.4 0.1]
 [5.9 2.1]
 [5.7 2.3]
 [5.8 2.2]
 [5.6 2.1]
 [1.6 0.2]
 [1.6 0.2]
 [5.1 2. ]
 [5.7 2.1]
 [1.3 0.3]
 [5.4 2.3]
 [1.4 0.2]
 [5.  2. ]
 [5.4 2.1]
 [1.3 0.2]
 [1.4 0.2]
 [5.8 1.6]
 [1.4 0.3]
 [1.3 0.2]
 [1.7 0.4]
 [4.  1.3]
 [5.9 2.3]
 [6.6 2.1]
 [1.4 0.2]
 [1.5 0.2]
 [1.4 0.2]
 [4.5 1.3]
 [4.4 1.4]
 [1.2 0.2]
 [1.7 0.5]
 [4.3 1.3]
 [1.5 0.4]
 [6.9 2.3]
 [3.3 1. ]
 [6.4 2. ]
 [4.4 1.4]
 [1.5 0.1]
 [4.8 1.8]
 [1.2 0.2]
 [6.7 2. ]
 [1.5 0.3]
 [1.6 0.2]
 [6.1 1.9]
 [1.4 0.2]
 [5.6 2.4]
 [4.1 1.3]
 [3.9 1.2]

In [4]:
from scikit_quri.circuit.pre_defined import create_qcl_ansatz,create_farhi_neven_ansatz
# circuit = create_farhi_neven_ansatz(n_qubits, 2,0)
circuit = create_qcl_ansatz(n_qubits,3,1.)
draw_circuit(circuit.circuit)
print(circuit.circuit.parameter_count)

   ___     ___     ___     ___     ___     ___     ___     ___     ___     ___  
  |PRY|   |PRZ|   |Mat|   |PRX|   |PRZ|   |PRX|   |Mat|   |PRX|   |PRZ|   |PRX| 
--|0  |---|1  |---|10 |---|11 |---|12 |---|13 |---|26 |---|27 |---|28 |---|29 |-
  |___|   |___|   |   |   |___|   |___|   |___|   |   |   |___|   |___|   |___| 
   ___     ___    |   |    ___     ___     ___    |   |    ___     ___     ___  
  |PRY|   |PRZ|   |   |   |PRX|   |PRZ|   |PRX|   |   |   |PRX|   |PRZ|   |PRX| 
--|2  |---|3  |---|   |---|14 |---|15 |---|16 |---|   |---|30 |---|31 |---|32 |-
  |___|   |___|   |   |   |___|   |___|   |___|   |   |   |___|   |___|   |___| 
   ___     ___    |   |    ___     ___     ___    |   |    ___     ___     ___  
  |PRY|   |PRZ|   |   |   |PRX|   |PRZ|   |PRX|   |   |   |PRX|   |PRZ|   |PRX| 
--|4  |---|5  |---|   |---|17 |---|18 |---|19 |---|   |---|33 |---|34 |---|35 |-
  |___|   |___|   |   |   |___|   |___|   |___|   |   |   |___|   |___|   |___| 
   ___     ___    |   |    _

In [5]:
print(f"{x_train[3]=}")
# init_params = [-0.6459906318187236, 3.5726052125794032, -3.7261666316598814, -6.381331778040319, 4.448677843102995, 3.4708870076700777, -5.070968615387754, -6.328852747092929]
# init_params =[4.002148315014479, 1.6951199159934145, 0.25744424357926954, 0.10384619671527331, 5.109927617709579, 5.735012432197602, 3.811604993109835, 4.583562073612696, 3.415696558991173, 5.8752333142921085, 5.126159064066656, 0.017206504032783308, 5.3872299529679575, 0.21102439329246717, 4.584560380312186]
init_params = [np.random.rand() for _ in range(circuit.n_learning_params)]
unbound_circuit = circuit.bind_input_and_parameters(x_train[0],init_params)
for gate in unbound_circuit.gates:
    print(gate.name,gate.params)
    if gate.name == "UnitaryMatrix":
        # print(gate.unitary_matrix)
        pass

x_train[3]=array([1.3, 0.2])
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
UnitaryMatrix ()
RX (0.03406311316923405,)
RZ (0.8586194453985747,)
RX (0.9647029561057711,)
RX (0.34477036144393614,)
RZ (0.2442035733327942,)
RX (0.5884849622664381,)
RX (0.5956681578097687,)
RZ (0.5951660880285304,)
RX (0.14330479187226342,)
RX (0.8019360302911975,)
RZ (0.7394432493684291,)
RX (0.35330436239915997,)
RX (0.2615571800303529,)
RZ (0.7517408353474151,)
RX (0.3133215799282546,)
UnitaryMatrix ()
RX (0.6444297005747643,)
RZ (0.4260866509966471,)
RX (0.1335300484313494,)
RX (0.8768528694712685,)
RZ (0.11304739712239709,)
RX (0.9600501370296327,)
RX (0.20767779890251203,)
RZ (0.143023254552477,)
RX (0.18239523593846674,)
RX (0.9186450605843863,)
RZ (0.10991072333858898,)
RX (0.3321238700070934,)
RX (0.590826663765523,)
RZ (0.43300576454275996,)
RX (0.41627744310909354,)
Unit

In [6]:
from quri_parts.core.state import quantum_state
param_circuit_state = quantum_state(n_qubits=5,circuit=circuit.circuit)
params = circuit.generate_bound_params(x_train[0],init_params)
circuit_state = param_circuit_state.bind_parameters(params)

from quri_parts.qulacs.estimator import create_qulacs_vector_estimator
from quri_parts.core.operator import Operator, pauli_label

op = Operator({pauli_label("Z0"): 1.0})
# estimator = create_qulacs_vector_estimator()
estimator = create_qulacs_vector_concurrent_estimator()
estimate = create_qulacs_vector_estimator()(op, circuit_state)
print(f"{estimate.value.real*2.2=}")

estimate.value.real*2.2=-0.17959462193577205


In [7]:
from scikit_quri.qnn.classifier import QNNClassifier

ops = []
for i in range(num_class):
    op = Operator({pauli_label(f"Z {i}"):1.0})
    ops.append(op)

adam = Adam()
estimator = create_qulacs_vector_concurrent_estimator()
gradient_estimator = create_parameter_shift_gradient_estimator(create_qulacs_vector_concurrent_parametric_estimator())
qnn = QNNClassifier(circuit,num_class,estimator,gradient_estimator,adam,ops,)

In [8]:
res = qnn._predict_inner(x_train,params=init_params)
print(res)

[[-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.05192095 -0.37551109 -0.14702704]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.13127142 -0.42798572  0.03685009]
 [-0.05192095 -0.37551109 -0.14702704]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.05192095 -0.37551109 -0.14702704]
 [-0.17959462 -0.10184819  0.08023379]
 [-0.17959462 -0.10184819

In [9]:
print(circuit.get_learning_param_indexes())
res = qnn.cost_func_grad(x_train,y_train,init_params)

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]


In [10]:
qnn.fit(x_train,y_train,maxiter=500)

init_params=array([3.32237746, 3.82264006, 2.15837718, 5.52449219, 3.74441536,
       1.63960572, 0.5528329 , 5.17628936, 2.9340449 , 5.17130076,
       2.95098109, 1.51068156, 4.16922945, 2.97034203, 4.67972865,
       4.61552886, 2.06067509, 0.8215488 , 4.66763115, 2.66905375,
       4.520552  , 4.80822521, 2.72465015, 0.42997889, 3.07819032,
       4.42641991, 0.12557147, 0.52652355, 6.019433  , 1.33607618,
       5.32797096, 2.06078564, 5.02512342, 6.08005835, 1.02762538,
       0.04767987, 1.11666255, 3.15855484, 3.46640699, 5.57427414,
       0.38350623, 0.61368972, 3.7168289 , 4.37664773, 4.80485298])
params[:4]=array([3.32237746, 3.82264006, 2.15837718, 5.52449219])
cost_func(init_params)=0.9877910168178473
self._estimate_grad(x_scaled,init_params)=array([[[-1.58395372e-01,  1.43430585e-01,  3.30453250e-01, ...,
          2.77555756e-17, -2.77555756e-17,  2.77555756e-17],
        [-5.94183431e-03, -4.19234074e-02, -5.72143834e-02, ...,
          2.77555756e-17,  0.00000000e+00,

KeyboardInterrupt: 

In [ ]:
y_pred = qnn.predict(x_test)
print(y_pred)
print(y_test)
from scipy.special import softmax
from sklearn.metrics import log_loss

y_pred_sm = softmax(y_pred,axis=1)
print(y_pred_sm)
log = log_loss(y_test,y_pred_sm)
print(log)

[[-1.00032576 -1.34144928 -0.2319797 ]
 [-0.90820358  0.53196672 -0.52652201]
 [ 0.27904822  0.25731059 -0.51303696]
 [-0.28550018  0.17303456  1.15248768]
 [ 0.21774731  0.31618236 -0.45274049]
 [-1.1503741  -0.6965795  -0.70923896]
 [ 0.35662233  0.14530502 -0.69633455]
 [-0.97280635  0.11705081 -0.08499152]
 [-0.97257435  0.18989272 -0.05976834]
 [-0.91594141  0.27755931 -0.46218168]
 [-0.76841235  0.21573364  0.54574387]
 [-0.96984339  0.12316847 -0.19858934]
 [-0.98208034  0.34730218 -0.19782887]
 [-0.97298319  0.12001232 -0.14391651]
 [-0.97571877  0.19229539 -0.12124181]
 [ 0.12842814  0.28161275 -0.09496872]
 [-0.96984339  0.12316847 -0.19858934]
 [-0.96641272  0.35311581 -0.34468643]
 [ 0.30289591  0.21256048 -0.62071676]
 [ 0.34899118  0.09664571 -0.74116183]
 [-0.95431051 -0.48444131  0.11900722]
 [-0.96984339  0.12316847 -0.19858934]
 [-0.04315306  0.50813761 -0.32938078]
 [ 0.27904822  0.25731059 -0.51303696]
 [-0.96197017 -0.1755576   0.05938136]
 [ 0.2952473  -0.07117232

d:\github\scikit-quri\pyenv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
